In [68]:
import pandas as pd
from typing import Literal

In [65]:
option = pd.read_pickle("C:\\PICKLE\\Nifty Options\\2019-03-07_nifty.pkl").set_index("date_time")
fut = pd.read_pickle("C:\\PICKLE\\Nifty Future\\2019-03-07_nifty_future.pkl").set_index("date_time")
df=pd.read_parquet(r"c:\Users\ashu4\Downloads\NIFTY 2022-01-03 NRE No-1.parquet")
option.index = pd.to_datetime(option.index)
fut.index = pd.to_datetime(fut.index)
option['date_time'] = option.index
fut['date_time'] = fut.index

In [66]:
df.to_csv("NIFTY 2022-01-03 NRE No-1.csv")

In [64]:
option.head()

,index,scrip,open,high,low,close,volume,openinterest,expiry_date,dte,date_time
date_time,,,,,,,,,,,
2019-03-07 09:15:00,NIFTY,10000PE,0.10,0.10,0.1,0.1,900,660750,2019-03-07,0,2019-03-07 09:15:00
2019-03-07 09:15:00,NIFTY,10400PE,0.15,0.15,0.1,0.1,120525,467775,2019-03-07,0,2019-03-07 09:15:00
2019-03-07 09:15:00,NIFTY,10500PE,0.20,0.25,0.1,0.1,27150,867375,2019-03-07,0,2019-03-07 09:15:00
2019-03-07 09:15:00,NIFTY,10550PE,0.20,0.30,0.1,0.1,3075,29925,2019-03-07,0,2019-03-07 09:15:00
2019-03-07 09:15:00,NIFTY,10600PE,0.45,0.45,0.1,0.1,65400,641775,2019-03-07,0,2019-03-07 09:15:00


In [63]:
entry_time = pd.to_datetime("2019-03-07 09:16:00")
exit_time = pd.to_datetime("2019-03-07 13:00:00")
current_dt="2019-03-07 09:15:00"


In [ ]:
def get_one_om(fut,future_price=None,step=50):
    future_price = fut['close'].iloc[0] if future_price is None else future_price
    return (int(future_price/step)*step)/100

def get_strangle_strike(fut,start_time,end_time,opt,gap=50,om=None,target=None,check_inverted=False, tf=1):

    valid_times = fut.loc[start_time:end_time].index
    for current_dt in valid_times:
        try:
            future_price = fut.loc[current_dt,'close']
            one_om = get_one_om(fut,future_price)
            target = one_om*om if target is None else target
            target_od = opt[(opt.index==current_dt) & (opt['close'] >= target*tf)].sort_values(by=['close']).copy()

            ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'),'scrip'].iloc[0]
            pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'), 'scrip'].iloc[0]

            ce_scrip_list = [ce_scrip, f"{int(ce_scrip[:-2])-gap}CE", f"{int(ce_scrip[:-2])+gap}CE"]
            pe_scrip_list = [pe_scrip, f"{int(pe_scrip[:-2])-gap}PE", f"{int(pe_scrip[:-2])+gap}PE"]

            call_list_prices, put_list_prices = [], []
            for z in range(3):
                try:
                    call_list_prices.append(opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip_list[z])]['close'].iloc[0])
                except:
                    call_list_prices.append(0)
                try:
                    put_list_prices.append(opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip_list[z])]['close'].iloc[0])
                except:
                    put_list_prices.append(0)

            call, put, min_diff = call_list_prices[0], put_list_prices[0], float('inf')
            target_2, target_3 = target*2*tf, target*3

            diff = abs(put-call)
            required_call, required_put = None, None
            if (put+call >= target_2) & (min_diff > diff) & (put+call <= target_3):
                min_diff = diff
                required_call, required_put = call, put            

            for i in range(1,3):
                if (min_diff > abs(put_list_prices[i] - call)) & (put_list_prices[i]+call >= target_2) & (put_list_prices[i]+call <= target_3):
                    min_diff = abs(put_list_prices[i] - call)
                    required_call, required_put = call, put_list_prices[i]
                if (min_diff > abs(call_list_prices[i] - put)) & (call_list_prices[i]+put >= target_2) & (call_list_prices[i]+put <= target_3):
                    min_diff = abs(call_list_prices[i] - put)
                    required_call, required_put = call_list_prices[i], put

            ce_scrip, pe_scrip = ce_scrip_list[call_list_prices.index(required_call)], pe_scrip_list[put_list_prices.index(required_put)]
            ce_price, pe_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip)]['close'].iloc[0], opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip)]['close'].iloc[0]
            
            if int(ce_scrip[:-2]) < int(pe_scrip[:-2]) and check_inverted:
                get_strangle_strike(fut,current_dt,end_time,opt,om)
            else:
                return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt


        except Exception as e  :
            print("error occured in get strangle strike",e)
            continue
    
    return None, None, None, None, None, None

def get_straddle_strike( start_dt, end_dt,opt,fut,gap):
    valid_times = fut.loc[start_dt:end_dt].index
    for current_dt in valid_times:
        try:
            future_price = fut.loc[current_dt,'close']
            round_future_price = round(future_price/gap)*gap
            ce_scrip, pe_scrip = f"{round_future_price}CE", f"{round_future_price}PE"
            ce_price, pe_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip)]['close'].iloc[0], opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip)]['close'].iloc[0]
            syn_future = ce_price - pe_price + round_future_price
            round_syn_future = round(syn_future/gap)*gap

            ce_scrip_list = [f"{round_syn_future}CE", f"{round_syn_future+gap}CE", f"{round_syn_future-gap}CE"]
            pe_scrip_list = [f"{round_syn_future}PE", f"{round_syn_future+gap}PE", f"{round_syn_future-gap}PE"]

            scrip_index=0

            for i in range(3):
                try:
                        ce_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip_list[i])]['close'].iloc[0]
                        pe_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip_list[i])]['close'].iloc[0]
                        # pe_price = opt.loc[(current_dt,pe_scrip_list[i]),'close']
                        diff = abs(ce_price-pe_price)
                        if min_value > diff:
                            min_value = diff
                            scrip_index = i
                except:
                        pass
            
            ce_scrip, pe_scrip = ce_scrip_list[scrip_index], pe_scrip_list[scrip_index]
            ce_price, pe_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip)]['close'].iloc[0], opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip)]['close'].iloc[0]

            return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt
        
        except (IndexError, KeyError, ValueError, TypeError):
            continue
        except Exception as e:
            print("get strike selection",e)
            continue
    return None, None, None, None, None, None

def run_leg(data:pd.DataFrame,sl:int, sl_price,method:Literal["HL","CC"], order_side:Literal['CE','PE']):
    """Runs one option leg (CE or PE) and returns metadata updates."""
    total_pnl = 0
    price = data.iloc[0].close


    data=data.iloc[1:]
    sl_count = 0

    if method == 'HL' :
        high='high'
        low='low'
    elif method == 'CC':
        high='close'
        low='close'

    leg_meta = {}
    for i in range(8):
        if i!=0:
            leg_meta[f"{order_side}{i}.Decay.Flag"] = False 
            leg_meta[f"{order_side}{i}.Decay.Time"] = ""
        leg_meta[f"{order_side}{i}.SL.Flag"] = False
        leg_meta[f"{order_side}{i}.SL.Time"] = ""
        leg_meta[f"{order_side}{i}.PNL"] = 0

    if sl == 0:

        pnl = price - data.iloc[-1].close - (0.01 * price)      
        leg_meta[f"{order_side}{0}.PNL"] = pnl
        
        return pnl , leg_meta


    while True:
        try:
            sl_time = data[data[high] >= sl_price].index[0]
        except:
            sl_time = None
        

        if sl_time is not None:
            if method == "HL":
                pnl = price - sl_price - (0.01 * price)
                total_pnl += pnl
            else:
                pnl = price - data.loc[sl_time,'close'] - (0.01 * price)
                total_pnl += pnl


            leg_meta[f"{order_side}{sl_count}.SL.Flag"] = True
            leg_meta[f"{order_side}{sl_count}.SL.Time"] = str(sl_time)
            leg_meta[f"{order_side}{sl_count}.PNL"] = pnl
            sl_count += 1

        if sl_time is None:

            if len(data.index)==0:
                print("No data left for exit, skipping leg.")
                break
            exit_time = data.index[-1]
            pnl = price - data.loc[exit_time].close - (0.01 * price)
            total_pnl += pnl

            
            leg_meta[f"{order_side}{sl_count}.PNL"] = pnl
            break

        try:
            sell_again_time = data[(data[low] <= price) & (data.index > sl_time)].index[0] 
        except:
            sell_again_time = None

        if sell_again_time is not None:
            data = data[data.index > sell_again_time]
            
            leg_meta[f"{order_side}{sl_count}.Decay.Flag"] = True
            leg_meta[f"{order_side}{sl_count}.Decay.Time"] = str(sell_again_time)
        else:
            break

    return total_pnl, leg_meta

def ner_both(option,future, strategy="NRE", symbol="NIFTY",
             start_time="09:20:00", end_time="15:25:00",
             method='CC', sl=20.0, resl=20.0, om=0,
             trade_date="2022-01-04",
             ):
    opt = option.loc[start_time:end_time]
    
    if om == 0:
        ce_scrip,pe_scrip,ce_price,pe_price,future_scrip,entry_time=get_straddle_strike(start_time,end_time,opt.copy(),future.copy(),50)
    else:
        ce_scrip,pe_scrip,ce_price,pe_price,future_scrip,entry_time=get_strangle_strike(fut.copy(),start_time,end_time,opt.copy(),om=om)
    
    ce_sl_price = ce_price* (1+sl/100)
    pe_sl_price = pe_price* (1+sl/100)

    ce_data = opt[(opt['scrip'] == ce_scrip) ]
    pe_data = opt[(opt['scrip'] == pe_scrip) ]
    ce_price = ce_data.loc[entry_time, 'close']
    pe_price = pe_data.loc[entry_time, 'close']

    ce_pnl, ce_meta = run_leg(ce_data,sl, ce_sl_price, method=method,order_side="CE")
    pe_pnl, pe_meta = run_leg(pe_data,sl, pe_sl_price, method=method,order_side="PE")

    metadata = {
        "P_Strategy": strategy,
        "P_Index": symbol,
        "P_StartTime": start_time,
        "P_EndTime": end_time,
        "P_OrderSide": "SELL",
        "P_Method": method,
        "P_SL": sl,
        "P_ReSL": resl,
        "P_OM": om,
        "Date": trade_date,
        "Day": pd.to_datetime(trade_date).day_name(),
        "DTE": 3,
        "EntryTime": start_time,
        "Future": future_scrip,
        "CE.Strike": ce_scrip,
        "CE.Price": ce_data.iloc[0].close,
        "CE.SL.Price": ce_sl_price if ce_data.iloc[0].close !=ce_sl_price else None
    }
    pe={
        "PE.Strike": pe_scrip,
        "PE.Price": pe_data.iloc[0].close,
        "PE.SL.Price": pe_sl_price if pe_data.iloc[0].close != pe_sl_price else None,
    }

    # Merge CE and PE legs
    metadata.update(ce_meta)
    metadata.update(pe)
    metadata.update(pe_meta)

    return ce_pnl + pe_pnl, metadata

In [11]:
start_dates = df.P_StartTime.unique()
end_times=df.P_EndTime.unique()
methods=df.P_Method.unique()    # HL , CC 
sls=resls=df.P_SL.unique()
oms=df.P_OM.unique()

In [ ]:
combinations=[]
for start_date in start_dates:
    for end_time in end_times:
        for method in methods:
            for sl in sls:
                for om in oms:
                    print(start_date,end_time,method,sl,om)
                    pnl , rows=ner_both(option.copy(),fut.copy(),start_time=start_date,end_time=end_time,
                        method=method,
                        sl=sl,resl=sl,
                        om=om,
                        trade_date=option.index[0].date()
                        )
                    combinations.append(rows)



In [ ]:
combination=pd.DataFrame(combinations)
combination.to_csv("nre_with_combination v6.csv")

In [ ]:
from pathlib import Path
import os
cwd = Path.cwd()
nifty_future = Path("C:\\PICKLE\\Nifty Future")
nifty_options = Path("C:\\PICKLE\\Nifty Options")
output_folder = nifty_options / "processed"
output_folder.mkdir(exist_ok=True) 

nifty_options_list = []
nifty_future_list = []

for file in nifty_options.iterdir():
    if file.is_file():
        # new_name = file.stem
        file_Path = nifty_options / file
        nifty_options_list.append(file_Path)
        # print(file_Path)
        
for file in nifty_future.iterdir():
    if file.is_file():
        file_Path = nifty_future / file
        nifty_future_list.append(file_Path)
        # print(file_Path)

df=pd.read_parquet(r"C:\Users\ashu4\Downloads\NIFTY 2019-03-07 NRE No-1.parquet") # for parameter selection 
    
processed_files = cwd / "processed_files"

processed_files.mkdir(exist_ok=True)


for future_path,option_path in zip(nifty_future_list,nifty_options_list):
    option = pd.read_pickle(option_path).set_index("date_time")
    fut = pd.read_pickle(future_path).set_index("date_time")
    option.index = pd.to_datetime(option.index)
    fut.index = pd.to_datetime(fut.index)
    option['date_time'] = option.index
    fut['date_time'] = fut.index

    start_dates = df.P_StartTime.unique()
    end_times=df.P_EndTime.unique()
    methods=df.P_Method.unique()    # HL , CC 
    sls=resls=df.P_SL.unique()
    oms=df.P_OM.unique()

    combinations=[]
    for start_date in start_dates:
        for end_time in end_times:
            # for method in methods:
            #     for sl in sls:
            #         for om in oms:
                        print(start_date,end_time,method,sl,om)
                        pnl , rows=ner_both(option.copy(),fut.copy(),start_time=start_date,end_time=end_time,
                            method=methods[0],
                            sl=sls[0],resl=sls[0],
                            om=oms[0],
                            trade_date=option.index[0].date()
                            )
                        combinations.append(rows)

    combination=pd.DataFrame(combinations)
    combination.to_csv(f"{os.path.join(processed_files,str(option.index[0].date())) }.csv")



In [32]:
nifty_options_list[0].stem

'2019-02-11_nifty'

In [ ]:
ce_scrip,pe_scrip,ce_price,pe_price,future_scrip,e_time=get_strangle_strike(fut,entry_time,exit_time,option,om=0.25)
ce_scrip,pe_scrip,ce_price,pe_price,future_scrip,e_time

In [ ]:
entry_time = pd.to_datetime("2019-03-07 09:16:00")
exit_time = pd.to_datetime("2019-03-07 15:25:00")
ce_sl_price = ce_price* (1+30/100)
pe_sl_price = pe_price* (1+30/100)
opt = option.loc[entry_time:exit_time].copy()
ce_data = opt[(opt['scrip'] == ce_scrip) ]
pe_data = opt[(opt['scrip'] == pe_scrip) ]
ce_price = ce_data.loc[entry_time, 'close']
pe_price = pe_data.loc[entry_time, 'close']

ce_pnl, ce_meta = run_leg(ce_data,30, ce_sl_price, method='CC',order_side="CE")
ce_price,ce_sl_price,"pnl",ce_pnl,ce_meta
pnl , rows=ner_both(opt.copy(),fut.copy(),start_time=start_dates[0],end_time=end_times[0],
                        method=methods[0],
                        sl=0,resl=0,
                        om=0,
                        trade_date=opt.index[0].date()
                        )
pnl

In [43]:
from pathlib import Path
import pandas as pd
from datetime import datetime

In [56]:
# Paths
nifty_future = Path("C:\\PICKLE\\Nifty Future")
nifty_options = Path("C:\\PICKLE\\Nifty Options")
param_df = pd.read_csv("parameter.csv")

In [57]:
# Read start and end date
start_date = pd.to_datetime(param_df.loc[0, "start_date"])
end_date   = pd.to_datetime(param_df.loc[0, "end_date"])

output_folder = nifty_options / "processed"
output_folder.mkdir(exist_ok=True) 

In [46]:
def get_file_date(file: Path) -> datetime:
    """
    Extract date from filename prefix: YYYY-MM-DD_xxx.pkl
    """
    try:
        date_str = file.stem.split("_")[0]  # e.g. "2019-03-07"
        return datetime.strptime(date_str, "%Y-%m-%d")
    except Exception:
        return None

In [58]:
start_date

Timestamp('2019-11-02 00:00:00')

In [59]:
options_dict = {}
for file in nifty_options.iterdir():
    if file.is_file():
        file_date = get_file_date(file)
        if file_date and start_date <= file_date <= end_date:
            options_dict[file_date.date()] = file

futures_dict = {}
for file in nifty_future.iterdir():
    if file.is_file():
        file_date = get_file_date(file)
        if file_date and start_date <= file_date <= end_date:
            futures_dict[file_date.date()] = file

In [61]:
# Keep only dates where both exist
common_dates = sorted(set(options_dict.keys()) & set(futures_dict.keys()))

nifty_options_list = [options_dict[d] for d in common_dates]
nifty_future_list  = [futures_dict[d] for d in common_dates]

print("Matched Dates:", common_dates)
print("Options Files:", nifty_options_list)
print("Futures Files:", nifty_future_list)


Matched Dates: [datetime.date(2019, 11, 4), datetime.date(2019, 11, 5), datetime.date(2019, 11, 6), datetime.date(2019, 11, 7), datetime.date(2019, 11, 8), datetime.date(2019, 11, 11), datetime.date(2019, 11, 13), datetime.date(2019, 11, 14), datetime.date(2019, 11, 15), datetime.date(2019, 11, 18), datetime.date(2019, 11, 19), datetime.date(2019, 11, 20), datetime.date(2019, 11, 21), datetime.date(2019, 11, 22), datetime.date(2019, 11, 25), datetime.date(2019, 11, 26), datetime.date(2019, 11, 27), datetime.date(2019, 11, 28), datetime.date(2019, 11, 29), datetime.date(2019, 12, 2), datetime.date(2019, 12, 3), datetime.date(2019, 12, 4), datetime.date(2019, 12, 5), datetime.date(2019, 12, 6), datetime.date(2019, 12, 9), datetime.date(2019, 12, 10), datetime.date(2019, 12, 11), datetime.date(2019, 12, 12), datetime.date(2019, 12, 13), datetime.date(2019, 12, 16), datetime.date(2019, 12, 17), datetime.date(2019, 12, 18), datetime.date(2019, 12, 19), datetime.date(2019, 12, 20), datetime.